In [62]:
import json
import os
import sys
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [63]:
path = str(Path().resolve() / 'agents/Group27/mcts')
if (path not in sys.path):
    sys.path.append(path)

from PolicyModel import PolicyModel

# Data

In [64]:
path = str(Path().resolve())
with open(os.path.join(path, '..', 'data', 'selfPlayData.json'), 'rb') as f:
    rawData = json.load(f)
print(len(rawData))

1086


In [65]:
for item in rawData.items():
    print(item[0])
    tempMoves = item[1]
    print(tempMoves)
    break

0 0 0 0 0 0 0 0 0 0 0 
 0 0 0 0 0 0 0 0 0 0 0 
  0 0 0 0 0 0 0 0 0 0 0 
   0 0 0 0 0 0 0 0 0 0 0 
    0 0 0 0 0 0 0 0 0 0 0 
     0 0 0 0 0 0 0 0 0 0 0 
      0 0 0 0 0 0 0 0 0 0 0 
       0 0 0 0 0 0 0 0 0 0 0 
        0 0 0 0 0 0 0 0 0 0 0 
         0 0 0 0 0 0 0 0 0 0 0 
          0 0 0 0 0 0 0 0 0 0 0 

[[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]


In [66]:
tempBoard = "R 0 0 0 R B 0 0 B R 0 \n R 0 B 0 0 B R B 0 R B \n  B 0 R 0 R R B 0 B R 0 \n   0 0 0 0 0 B 0 R 0 B 0 \n    B R 0 0 B R B R R R R \n     R B B B 0 0 R B 0 0 R \n      B R 0 R 0 R R B R B 0 \n       B R R B R B B B 0 R R \n        B R 0 0 B 0 0 0 R B 0 \n         0 B R B B B R B B 0 R \n          B 0 0 0 0 B R B R R R \n"

## State Tensors

In [67]:
def tensorfyBoard(boardString):
    boardString = boardString.replace('R', '1').replace('B', '2')
    boardRows = boardString.strip().split('\n')
    board = [list(map(int, row.strip().split())) for row in boardRows]

    board = torch.tensor(board, dtype=torch.int)

    rStones = (board == 1).int()
    bStones = (board == 2).int()
    nStones = (board == 0).int()

    return torch.stack([rStones, bStones, nStones])

tensorfyBoard(tempBoard)

tensor([[[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
         [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
         [0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1],
         [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
         [0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0],
         [0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1],
         [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]],

        [[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1],
         [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
         [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
         [0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
         [1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0],
         [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
         [0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0],
         [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]],

        [[0, 1, 1, 1, 0, 0, 

## Move Frequencies

In [68]:
def encodeMoves(moves2D, boardTensor):
    moves = torch.tensor(np.array(moves2D).flatten())

    # smoothing
    moves += boardTensor[2].flatten()

    # normalise
    moves = moves / moves.sum()
    return moves

encodeMoves(tempMoves, tensorfyBoard(tempBoard))

tensor([0.0000, 0.0156, 0.0156, 0.0312, 0.0156, 0.0156, 0.0156, 0.0156, 0.0000,
        0.0000, 0.0156, 0.0000, 0.0156, 0.0000, 0.0156, 0.0156, 0.0000, 0.0156,
        0.0000, 0.0156, 0.0000, 0.0000, 0.0000, 0.0156, 0.0000, 0.0156, 0.0000,
        0.0000, 0.0156, 0.0312, 0.0156, 0.0000, 0.0156, 0.0312, 0.0156, 0.0156,
        0.0156, 0.0156, 0.0000, 0.0156, 0.0000, 0.0156, 0.0000, 0.0156, 0.0000,
        0.0000, 0.0469, 0.0312, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0156, 0.0000, 0.0000, 0.0000, 0.0156, 0.0156, 0.0000, 0.0000,
        0.0156, 0.0156, 0.0000, 0.0000, 0.0156, 0.0156, 0.0000, 0.0156, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0156, 0.0156, 0.0156, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0469, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0156, 0.0156, 0.0000, 0.0156, 0.0156, 0.0156, 0.0000, 0.0000, 0.0156,
        0.0312, 0.0000, 0.0156, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0156, 0.0000, 0.0000, 0.0156, 

## ...

In [69]:
def processDataSet(dataSet):
    boardTensors = []
    moveTensors = []

    for (boardString, moves2D) in dataSet.items():
        boardTensor = tensorfyBoard(boardString)
        moveTensor = encodeMoves(moves2D, boardTensor)

        boardTensors.append(boardTensor)
        moveTensors.append(moveTensor)

    boards = torch.stack(boardTensors)
    moves = torch.stack(moveTensors)

    return boards, moves

boards, moves = processDataSet(rawData)
print(boards.shape, moves.shape)

torch.Size([1086, 3, 11, 11]) torch.Size([1086, 121])


# Data Augmentation

The Hex board is symmetric. We can use this to augment our data. For each board state, we can generate 6 more board states by rotating the board by 60 degrees each time. This will give us 7 times more data to train on.

In [ ]:
# TODO

# Training

In [70]:
model = PolicyModel(boardSize=11)

In [71]:
def trainModel(model, boards, moves, learningRate=1e-3, epochs=10):

    optimiser = optim.Adam(model.parameters(), lr=learningRate)
    criterion = torch.nn.KLDivLoss(reduction='batchmean')

    model.train()

    for epoch in range(epochs):
        optimiser.zero_grad()

        # forward
        logits = model(boards)
        logProbs = torch.nn.functional.log_softmax(logits, dim=1)

        # loss
        loss = criterion(logProbs, moves)

        # backpropagation
        loss.backward()
        optimiser.step()

        print(f'Epoch {epoch+1}: {loss.item()}')

    return model

trainModel(model, boards.float(), moves)

Epoch 1: 0.784312903881073
Epoch 2: 0.7817811965942383
Epoch 3: 0.7784368991851807
Epoch 4: 0.7742353677749634
Epoch 5: 0.7701846361160278
Epoch 6: 0.7666717171669006
Epoch 7: 0.7622544765472412
Epoch 8: 0.7573235034942627
Epoch 9: 0.7527790665626526
Epoch 10: 0.7484581470489502


PolicyModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (convp): Conv2d(64, 2, kernel_size=(1, 1), stride=(1, 1))
  (fcp): Linear(in_features=242, out_features=121, bias=True)
)

# Results

In [72]:
def infer(model, boardString):
    model.eval()
    with torch.no_grad():
        boardTensor = tensorfyBoard(boardString)
        logits = model(boardTensor.float().unsqueeze(0))
        probs = torch.nn.functional.softmax(logits, dim=1)

    return probs

infer(model, tempBoard)

tensor([[0.0093, 0.0084, 0.0081, 0.0079, 0.0059, 0.0067, 0.0088, 0.0062, 0.0105,
         0.0072, 0.0063, 0.0050, 0.0068, 0.0059, 0.0097, 0.0055, 0.0065, 0.0119,
         0.0078, 0.0050, 0.0083, 0.0088, 0.0095, 0.0097, 0.0072, 0.0075, 0.0088,
         0.0089, 0.0076, 0.0103, 0.0141, 0.0080, 0.0080, 0.0137, 0.0068, 0.0128,
         0.0074, 0.0100, 0.0093, 0.0091, 0.0073, 0.0085, 0.0063, 0.0129, 0.0075,
         0.0084, 0.0049, 0.0060, 0.0084, 0.0116, 0.0059, 0.0079, 0.0089, 0.0061,
         0.0107, 0.0058, 0.0119, 0.0063, 0.0071, 0.0055, 0.0089, 0.0085, 0.0062,
         0.0078, 0.0052, 0.0087, 0.0076, 0.0102, 0.0053, 0.0081, 0.0137, 0.0088,
         0.0082, 0.0088, 0.0086, 0.0091, 0.0070, 0.0079, 0.0116, 0.0069, 0.0081,
         0.0084, 0.0066, 0.0055, 0.0095, 0.0073, 0.0140, 0.0089, 0.0102, 0.0070,
         0.0057, 0.0088, 0.0071, 0.0081, 0.0054, 0.0132, 0.0089, 0.0119, 0.0066,
         0.0068, 0.0060, 0.0085, 0.0077, 0.0113, 0.0102, 0.0043, 0.0076, 0.0053,
         0.0057, 0.0084, 0.0

In [73]:
torch.save(model.state_dict(), './models/test.pth')